In [246]:
import numpy as np
import scipy as sp
import gridData as gd
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import pytraj as pt
import py3Dmol as pmol

import sys
import os
import tqdm
import gc
import copy

## Testing py3Dmol for molecule viewing

In [5]:
p = pmol.view(query='pdb:1ycr')
p.setStyle({'cartoon': {'color':'spectrum'}})
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## testing plotly for interactive 3D rendering

In [34]:
#Toy test system to test rendering of 3d scatter plots with plotly
pointsList=np.array([
    np.array([0.,0.,10.])+np.random.random(3),
    np.array([0.,0.,-10.])+np.random.random(3),
    np.array([1.,0.,0.])*5.+np.random.random(3),
    np.array([np.cos(120.*np.pi/180.),np.sin(120.*np.pi/180.),0])*5.+np.random.random(3),
    np.array([np.cos(-120*np.pi/180.),np.sin(-120.*np.pi/180.),0])*5.+np.random.random(3)
])
pointsFrame=pd.DataFrame(
    {'X':pointsList[:,0],
     'Y':pointsList[:,1],
     'Z':pointsList[:,2]})
pointsFrame

,X,Y,Z
0,0.016234,0.870661,10.852613
1,0.317801,0.349861,-9.431428
2,5.722099,0.772032,0.885702
3,-2.491582,5.272158,0.589948
4,-1.776874,-4.041210,0.345971


In [26]:
import plotly as ply

In [27]:
ply.io.renderers.default="notebook"

In [35]:
go=ply.graph_objs
fig=go.Figure(data=[go.Scatter3d(x=pointsFrame['X'],y=pointsFrame['Y'],z=pointsFrame['Z'],
                                mode='markers')])
fig.show()

# Load Coordinate data sets

In [38]:
dataDir="simulationData"

potFilePattern="pot_"
potDataFiles=[fileName for fileName in os.listdir(dataDir) \
              if potFilePattern in fileName]
print("potassium data file list:",potDataFiles)

print()
alphaFilePattern="atomCA_"
alphaDataFiles=[fileName for fileName in os.listdir(dataDir) \
                if alphaFilePattern in fileName] 
print("alpha carbon control atom data file list:",alphaDataFiles)

potassium data file list: ['pot_165552.dat', 'pot_165627.dat', 'pot_165623.dat', 'pot_165554.dat', 'pot_165581.dat', 'pot_165557.dat', 'pot_165530.dat', 'pot_165872.dat', 'pot_165711.dat', 'pot_165891.dat', 'pot_166052.dat', 'pot_165570.dat', 'pot_166069.dat', 'pot_166136.dat', 'pot_166095.dat', 'pot_165774.dat']

alpha carbon control atom data file list: ['atomCA_resid2377.dat', 'atomCA_resid1178.dat', 'atomCA_resid3796.dat', 'atomCA_resid958.dat', 'atomCA_resid1366.dat']


In [43]:
?pd.read_csv

In [92]:
atomDataFrames=[]
for potFile in tqdm.tqdm_notebook(potDataFiles,desc='Loading Potassium Data'):
    potID=potFile.split('.')[0].split('_')[-1]
    potFilePath='/'.join([dataDir,potFile])
    tempFrame=pd.read_csv(potFilePath,delim_whitespace=True,skiprows=1,
                          names=['Frame','X','Y','Z','Unused1','Unused2','Unused3'])
    tempFrame=tempFrame.drop(columns=[colName for colName in tempFrame.columns \
                                      if 'Unused' in colName])
    tempFrame['AtomID']=potID
    tempFrame['AtomType']='POT'
    tempFrame=tempFrame[['AtomID','AtomType','Frame','X','Y','Z']]
    atomDataFrames.append(tempFrame.copy())
    
for alphaFile in tqdm.tqdm_notebook(alphaDataFiles,'Loading Alpha Carbon Data'):
    alphaID=alphaFile.split('.')[0].split('_')[-1].replace('resid','')
    alphaFilePath='/'.join([dataDir,alphaFile])
    tempFrame=pd.read_csv(alphaFilePath,delim_whitespace=True,skiprows=1,
                          names=['Frame','X','Y','Z','Unused1','Unused2','Unused3'])
    tempFrame=tempFrame.drop(columns=[colName for colName in tempFrame.columns \
                                      if 'Unused' in colName])
    tempFrame['AtomID']=alphaID
    tempFrame['AtomType']='CA'
    tempFrame=tempFrame[['AtomID','AtomType','Frame','X','Y','Z']]
    atomDataFrames.append(tempFrame.copy())
    
atomDataFrame=pd.concat(atomDataFrames)
atomDataFrames=[]
del(atomDataFrames)
atomDataFrame.head()

,AtomID,AtomType,Frame,X,Y,Z
0,165552,POT,1,109.3891,81.3629,25.0487
1,165552,POT,2,105.4785,241.4059,92.7848
2,165552,POT,3,106.9650,240.9525,91.4489
3,165552,POT,4,106.4484,240.0916,91.4920
4,165552,POT,5,105.1581,240.6961,91.8088


In [53]:
atomDataFrame.AtomType.unique()

array(['POT', 'CA'], dtype=object)

In [93]:
atomDataFrame.to_csv('/'.join([dataDir,'atom_coordinate_data_table.csv']),
                     index=False)

## Visualize Loaded Atom Data as 3D scatter plot

In [94]:
atomDataFrame=pd.read_csv('/'.join([dataDir,'atom_coordinate_data_table.csv']))
atomDataFrame.head()

,AtomID,AtomType,Frame,X,Y,Z
0,165552,POT,1,109.3891,81.3629,25.0487
1,165552,POT,2,105.4785,241.4059,92.7848
2,165552,POT,3,106.9650,240.9525,91.4489
3,165552,POT,4,106.4484,240.0916,91.4920
4,165552,POT,5,105.1581,240.6961,91.8088


In [79]:
go=ply.graph_objs
fig=go.Figure(
    data=[
            go.Scatter3d(x=atomDataFrame.query('AtomType == "POT"')['X'],
                         y=atomDataFrame.query('AtomType == "POT"')['Y'],
                         z=atomDataFrame.query('AtomType == "POT"')['Z'],
                         mode='markers',
                         marker=dict(size=2,
                                     color='blue',
                                     colorscale='Viridis',
                                     opacity=.125)),
            go.Scatter3d(x=atomDataFrame.query('AtomType == "CA"')['X'],
                         y=atomDataFrame.query('AtomType == "CA"')['Y'],
                         z=atomDataFrame.query('AtomType == "CA"')['Z'],
                         mode='markers',
                         marker=dict(size=12,color='red',opacity=.75))
         ],
    layout=dict(width=800,height=600)
)
fig.show()

# Compute Potassium to Alpha Carbon Distances

In [159]:
tempFrames=[]
baseFrame=atomDataFrame.set_index('Frame')
for groupName,groupData in \
    tqdm.tqdm_notebook(atomDataFrame.query('AtomType == "CA"').groupby('AtomID'),
                       desc='Computing Distances'):
    tempFrame=baseFrame.join(groupData.set_index('Frame')[['X','Y','Z']],
                             rsuffix='_ref',how='outer')
    tempFrame['RefAtomID']=groupName
    tempFrame['Distance']=tempFrame[['X','Y','Z','X_ref','Y_ref','Z_ref']].apply(
        lambda x: np.sqrt(np.sum(
            [(x[0]-x[3])**2,
             (x[1]-x[4])**2,
             (x[2]-x[5])**2])),
        axis=1)
    tempFrames.append(tempFrame.copy())

diffData=pd.concat(tempFrames)
tempFrames=[]
del(tempFrames)
gc.collect()
diffData.head()

,AtomID,AtomType,X,Y,Z,X_ref,Y_ref,Z_ref,RefAtomID,Distance
Frame,,,,,,,,,,
1,165552,POT,109.3891,81.3629,25.0487,123.2928,110.994,58.3107,958,46.665572
1,165627,POT,32.3418,165.3300,37.5258,123.2928,110.994,58.3107,958,107.965260
1,165623,POT,187.0743,90.5251,102.1417,123.2928,110.994,58.3107,958,80.051310
1,165554,POT,42.9653,78.8911,51.0144,123.2928,110.994,58.3107,958,86.812093
1,165581,POT,57.8748,122.7727,107.9835,123.2928,110.994,58.3107,958,82.979754


In [162]:
diffCoords=diffData.drop(
        columns=[colName for colName in diffData.columns if 'ref' in colName]
    ).reset_index().pivot_table(index=['Frame','AtomID','AtomType','X','Y','Z'],
                                columns='RefAtomID',values='Distance')
diffCoords.columns=np.array(diffCoords.columns.map(lambda x: 'D_%g'%(x)))
diffCoords=diffCoords.reset_index()
diffCoords.head()

,Frame,AtomID,AtomType,X,Y,Z,D_958,D_1178,D_1366,D_2377,D_3796
0,1,958,CA,123.2928,110.9940,58.3107,0.000000,92.177611,41.136629,70.165424,63.568180
1,1,1178,CA,76.6708,159.4728,121.3417,92.177611,0.000000,96.855696,79.014331,72.235503
2,1,1366,CA,95.1964,126.0866,32.3294,41.136629,96.855696,0.000000,51.106721,51.621998
3,1,2377,CA,53.3092,115.8309,59.7553,70.165424,79.014331,51.106721,0.000000,75.214944
4,1,3796,CA,103.7853,171.4280,55.4642,63.568180,72.235503,51.621998,75.214944,0.000000


In [163]:
diffCoords.to_csv('/'.join([dataDir,'potassium_distance_data.csv']),
                  index=False)

# Visualize Potassium to Alpha Carbon Distances

We will use an ipython widget to allow the user to select which alpha carbon to use for colormapping the
potassium to alpha carbon distances.

In [164]:
diffCoords=pd.read_csv('/'.join([dataDir,'potassium_distance_data.csv']))
diffCoords.head()

,Frame,AtomID,AtomType,X,Y,Z,D_958,D_1178,D_1366,D_2377,D_3796
0,1,958,CA,123.2928,110.9940,58.3107,0.000000,92.177611,41.136629,70.165424,63.568180
1,1,1178,CA,76.6708,159.4728,121.3417,92.177611,0.000000,96.855696,79.014331,72.235503
2,1,1366,CA,95.1964,126.0866,32.3294,41.136629,96.855696,0.000000,51.106721,51.621998
3,1,2377,CA,53.3092,115.8309,59.7553,70.165424,79.014331,51.106721,0.000000,75.214944
4,1,3796,CA,103.7853,171.4280,55.4642,63.568180,72.235503,51.621998,75.214944,0.000000


In [174]:
diffCoords.sort_values(['AtomID','Frame']).tail()

,Frame,AtomID,AtomType,X,Y,Z,D_958,D_1178,D_1366,D_2377,D_3796
43616,2077,166136,POT,80.8816,223.3779,20.3468,127.960128,112.182235,96.202377,112.633095,62.610885
43637,2078,166136,POT,88.1827,216.8769,20.8253,119.425785,108.561632,88.875757,108.786181,54.407042
43658,2079,166136,POT,85.9101,213.5211,22.8924,116.520063,105.470168,85.208274,104.386001,51.314128
43679,2080,166136,POT,87.5495,208.5846,24.3888,110.868936,102.451812,80.666047,99.734712,47.000275
43700,2081,166136,POT,91.2438,200.8141,23.9601,103.452520,99.975768,72.538005,94.789927,40.659838


In [132]:
import ipywidgets as widgets
from ipywidgets import interact

In [176]:
@interact
def color_by_distance(distanceColumn=[colName for colName in diffCoords.columns \
                                      if 'D_' in colName]):
    go=ply.graph_objs
    fig=go.Figure(
        data=[
                go.Scatter3d(x=diffCoords.query('AtomType == "POT"')['X'],
                             y=diffCoords.query('AtomType == "POT"')['Y'],
                             z=diffCoords.query('AtomType == "POT"')['Z'],
                             mode='markers',
                             marker=dict(size=2,
                                         color=diffCoords.query(
                                                 'AtomType == "POT"'
                                             )[distanceColumn],
                                         colorscale='RdBu',
                                         opacity=.125)),
                go.Scatter3d(
                    x=diffCoords.query(
                            '(AtomType == "CA") and (AtomID != %s)'%(distanceColumn.split('_')[-1])
                        )['X'],
                    y=diffCoords.query(
                            '(AtomType == "CA") and (AtomID != %s)'%(distanceColumn.split('_')[-1])
                        )['Y'],
                    z=diffCoords.query(
                            '(AtomType == "CA") and (AtomID != %s)'%(distanceColumn.split('_')[-1])
                        )['Z'],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25)),
                go.Scatter3d(x=diffCoords.query('AtomID == %s'%(distanceColumn.split('_')[-1]))['X'],
                             y=diffCoords.query('AtomID == %s'%(distanceColumn.split('_')[-1]))['Y'],
                             z=diffCoords.query('AtomID == %s'%(distanceColumn.split('_')[-1]))['Z'],
                             mode='markers',
                             marker=dict(size=12,color='red',opacity=.75))
             ],
        layout=dict(width=800,height=600)
    )
    fig.show()

interactive(children=(Dropdown(description='distanceColumn', options=('D_958', 'D_1178', 'D_1366', 'D_2377', '…

In [145]:
import sklearn as skl
from sklearn.decomposition import PCA

In [177]:
Xdata=diffCoords.query('AtomType == "CA"')[diffCoords.columns[-5:]]
Xdata.head()

,D_958,D_1178,D_1366,D_2377,D_3796
0,0.000000,92.177611,41.136629,70.165424,63.568180
1,92.177611,0.000000,96.855696,79.014331,72.235503
2,41.136629,96.855696,0.000000,51.106721,51.621998
3,70.165424,79.014331,51.106721,0.000000,75.214944
4,63.568180,72.235503,51.621998,75.214944,0.000000


In [179]:
pca=PCA()

pca.fit(Xdata)

pca_coords=pca.transform(diffCoords[diffCoords.columns[-5:]])
pca_coords[:10]

array([[-43.64993096, -19.31983325,  40.04540602, -10.08752709,
         -2.22027533],
       [ 93.72146413,   5.13019504,  18.75433947,   7.83224295,
          0.66626237],
       [-52.80985461,  -2.93852585,  -9.72919662,  22.74829165,
         -0.14161154],
       ...,
       [ -3.86563455,  55.26846595,  11.17438836, -11.44833742,
         62.13011598],
       [ 19.13449053,  45.85510982,  31.67298277, -13.55090974,
         69.70756121],
       [-33.13771867,  10.50842811, -16.78598214,   9.5049671 ,
         21.22223154]])

In [180]:
pcaData=diffCoords.join(
    pd.DataFrame(pca_coords,
                 columns=['PCA_%g'%ii for ii in np.arange(pca_coords.shape[1])]))
pcaData.head()

,Frame,AtomID,AtomType,X,Y,Z,D_958,D_1178,D_1366,D_2377,D_3796,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4
0,1,958,CA,123.2928,110.9940,58.3107,0.000000,92.177611,41.136629,70.165424,63.568180,-43.649931,-19.319833,40.045406,-10.087527,-2.220275
1,1,1178,CA,76.6708,159.4728,121.3417,92.177611,0.000000,96.855696,79.014331,72.235503,93.721464,5.130195,18.754339,7.832243,0.666262
2,1,1366,CA,95.1964,126.0866,32.3294,41.136629,96.855696,0.000000,51.106721,51.621998,-52.809855,-2.938526,-9.729197,22.748292,-0.141612
3,1,2377,CA,53.3092,115.8309,59.7553,70.165424,79.014331,51.106721,0.000000,75.214944,-9.582098,58.857774,-14.499958,-12.334822,-0.773672
4,1,3796,CA,103.7853,171.4280,55.4642,63.568180,72.235503,51.621998,75.214944,0.000000,6.237899,-44.959842,-33.877175,-11.661829,-0.907584


In [181]:
pcaData.to_csv('/'.join([dataDir,'pca_projection_data.csv']),
                  index=False)

# Visualize PCA projection data

Lets see how well the first 3 principal components do at reconstructing our data set.

In [189]:
from ipywidgets import interact_manual

In [183]:
pcaData=pd.read_csv('/'.join([dataDir,'pca_projection_data.csv']))
pcaData.head()

,Frame,AtomID,AtomType,X,Y,Z,D_958,D_1178,D_1366,D_2377,D_3796,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4
0,1,958,CA,123.2928,110.9940,58.3107,0.000000,92.177611,41.136629,70.165424,63.568180,-43.649931,-19.319833,40.045406,-10.087527,-2.220275
1,1,1178,CA,76.6708,159.4728,121.3417,92.177611,0.000000,96.855696,79.014331,72.235503,93.721464,5.130195,18.754339,7.832243,0.666262
2,1,1366,CA,95.1964,126.0866,32.3294,41.136629,96.855696,0.000000,51.106721,51.621998,-52.809855,-2.938526,-9.729197,22.748292,-0.141612
3,1,2377,CA,53.3092,115.8309,59.7553,70.165424,79.014331,51.106721,0.000000,75.214944,-9.582098,58.857774,-14.499958,-12.334822,-0.773672
4,1,3796,CA,103.7853,171.4280,55.4642,63.568180,72.235503,51.621998,75.214944,0.000000,6.237899,-44.959842,-33.877175,-11.661829,-0.907584


In [203]:
@interact_manual
def color_by_distance(colorColumn=widgets.Dropdown(
                          options=[colName for colName in pcaData.columns \
                                   if ('PCA_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='D_958'),
                      xCol=widgets.Dropdown(
                          options=[colName for colName in pcaData.columns \
                                   if ('PCA_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='PCA_0'),
                      yCol=widgets.Dropdown(
                          options=[colName for colName in pcaData.columns \
                                   if ('PCA_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='PCA_1'),
                      zCol=widgets.Dropdown(
                          options=[colName for colName in pcaData.columns \
                                   if ('PCA_' in colName) | ('D_' in colName) | \
                                      (colName=='X') | (colName=='Y') | (colName=='Z')],
                          value='PCA_2'),
                     ):
    go=ply.graph_objs
    fig=go.Figure(
        data=[
                go.Scatter3d(x=pcaData.query('AtomType == "POT"')[xCol],
                             y=pcaData.query('AtomType == "POT"')[yCol],
                             z=pcaData.query('AtomType == "POT"')[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color=pcaData.query(
                                                 'AtomType == "POT"'
                                             )[colorColumn],
                                         colorscale='RdBu',
                                         opacity=.125)),
                go.Scatter3d(
                    x=pcaData.query(
                            '(AtomType == "CA")'
                        )[xCol],
                    y=pcaData.query(
                            '(AtomType == "CA")'
                        )[yCol],
                    z=pcaData.query(
                            '(AtomType == "CA")'
                        )[zCol],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25))
             ],
        layout=dict(
            width=800,height=600)
    )
    fig.show()

interactive(children=(Dropdown(description='colorColumn', index=3, options=('X', 'Y', 'Z', 'D_958', 'D_1178', …

In [283]:
@interact_manual
def testingFun( #'D_958', 'D_1178', 'D_1366', 'D_2377', 'D_3796'
    X_bound=widgets.FloatRangeSlider(     min=pcaData.X.min(),max=pcaData.X.max(),
                                   value=(pcaData.X.min(),pcaData.X.max())),
    
    Y_bound=widgets.FloatRangeSlider(     min=pcaData.Y.min(),max=pcaData.Y.max(),
                                   value=(pcaData.Y.min(),pcaData.Y.max())),
    
    Z_bound=widgets.FloatRangeSlider(     min=pcaData.Z.min(),max=pcaData.Z.max(),
                                   value=(pcaData.Z.min(),pcaData.Z.max())),
    
    D_958_bound=widgets.FloatRangeSlider( min=pcaData.D_958.min(),max=pcaData.D_958.max(),
                                   value=(pcaData.D_958.min(),pcaData.D_958.max())),
    
    D_1178_bound=widgets.FloatRangeSlider(min=pcaData.D_1178.min(),max=pcaData.D_1178.max(),
                                   value=(pcaData.D_1178.min(),pcaData.D_1178.max())),
    
    D_1366_bound=widgets.FloatRangeSlider(min=pcaData.D_1366.min(),max=pcaData.D_1366.max(),
                                   value=(pcaData.D_1366.min(),pcaData.D_1366.max())),
    
    D_2377_bound=widgets.FloatRangeSlider(min=pcaData.D_2377.min(),max=pcaData.D_2377.max(),
                                   value=(pcaData.D_2377.min(),pcaData.D_2377.max())),
    
    D_3796_bound=widgets.FloatRangeSlider(min=pcaData.D_3796.min(),max=pcaData.D_3796.max(),
                                   value=(pcaData.D_3796.min(),pcaData.D_3796.max())),
    
    PCA_0_bound=widgets.FloatRangeSlider( min=pcaData.PCA_0.min(),max=pcaData.PCA_0.max(),
                                   value=(pcaData.PCA_0.min(),pcaData.PCA_0.max())),
    
    PCA_1_bound=widgets.FloatRangeSlider( min=pcaData.PCA_1.min(),max=pcaData.PCA_1.max(),
                                   value=(pcaData.PCA_1.min(),pcaData.PCA_1.max())),
    
    PCA_2_bound=widgets.FloatRangeSlider( min=pcaData.PCA_1.min(),max=pcaData.PCA_1.max(),
                                   value=(pcaData.PCA_1.min(),pcaData.PCA_1.max())),
    
    PCA_3_bound=widgets.FloatRangeSlider( min=pcaData.PCA_3.min(),max=pcaData.PCA_3.max(),
                                   value=(pcaData.PCA_3.min(),pcaData.PCA_3.max())),
    
    PCA_4_bound=widgets.FloatRangeSlider( min=pcaData.PCA_4.min(),max=pcaData.PCA_4.max(),
                                   value=(pcaData.PCA_4.min(),pcaData.PCA_4.max())),
    xCol=widgets.Dropdown(options=pcaData.columns[3:],value='X'),
    yCol=widgets.Dropdown(options=pcaData.columns[3:],value='Y'),
    zCol=widgets.Dropdown(options=pcaData.columns[3:],value='Z'),
    ):
    kwargDict=dict(locals())
    print(kwargDict.keys(),kwargDict.values())
    kwargDict=dict(locals())
    kwargKeys=np.array(list(kwargDict.keys()))[:13]
    for kwargKey in kwargKeys:
        print(kwargKey,end="")
        print(len(kwargDict[kwargKey]),kwargDict[kwargKey][0],kwargDict[kwargKey][1])
    queryStr=' and '.join(
        ['({factorKey:s} > {factorMin:f}) and ({factorKey:s} < {factorMax:f})'.format(
            factorKey='_'.join(kwargKey.split('_')[:-1]),
            factorMin=kwargDict[kwargKey][0],
            factorMax=kwargDict[kwargKey][1]) \
         for kwargKey in kwargKeys])
    print("query:",queryStr)
    print("selection shape:",pcaData.query(queryStr).shape)

interactive(children=(FloatRangeSlider(value=(0.0063, 196.48), description='X_bound', max=196.48, min=0.0063),…

In [288]:
@interact_manual
def explore_milestone_bounds( #'D_958', 'D_1178', 'D_1366', 'D_2377', 'D_3796'
    X_bound=widgets.FloatRangeSlider(     min=pcaData.X.min()-.1,max=pcaData.X.max()+.1,
                                   value=(pcaData.X.min()-.1,pcaData.X.max()+.1)),
    
    Y_bound=widgets.FloatRangeSlider(     min=pcaData.Y.min()-.1,max=pcaData.Y.max()+.1,
                                   value=(pcaData.Y.min()-.1,pcaData.Y.max()+.1)),
    
    Z_bound=widgets.FloatRangeSlider(     min=pcaData.Z.min()-.1,max=pcaData.Z.max()+.1,
                                   value=(pcaData.Z.min()-.1,pcaData.Z.max()+.1)),
    
    D_958_bound=widgets.FloatRangeSlider( min=pcaData.D_958.min()-.1,max=pcaData.D_958.max()+.1,
                                   value=(pcaData.D_958.min()-.1,pcaData.D_958.max()+.1)),
    
    D_1178_bound=widgets.FloatRangeSlider(min=pcaData.D_1178.min()-.1,max=pcaData.D_1178.max()+.1,
                                   value=(pcaData.D_1178.min()-.1,pcaData.D_1178.max()+.1)),
    
    D_1366_bound=widgets.FloatRangeSlider(min=pcaData.D_1366.min()-.1,max=pcaData.D_1366.max()+.1,
                                   value=(pcaData.D_1366.min()-.1,pcaData.D_1366.max()+.1)),
    
    D_2377_bound=widgets.FloatRangeSlider(min=pcaData.D_2377.min()-.1,max=pcaData.D_2377.max()+.1,
                                   value=(pcaData.D_2377.min()-.1,pcaData.D_2377.max()+.1)),
    
    D_3796_bound=widgets.FloatRangeSlider(min=pcaData.D_3796.min()-.1,max=pcaData.D_3796.max()+.1,
                                   value=(pcaData.D_3796.min()-.1,pcaData.D_3796.max()+.1)),
    
    PCA_0_bound=widgets.FloatRangeSlider( min=pcaData.PCA_0.min()-.1,max=pcaData.PCA_0.max()+.1,
                                   value=(pcaData.PCA_0.min()-.1,pcaData.PCA_0.max()+.1)),
    
    PCA_1_bound=widgets.FloatRangeSlider( min=pcaData.PCA_1.min()-.1,max=pcaData.PCA_1.max()+.1,
                                   value=(pcaData.PCA_1.min()-.1,pcaData.PCA_1.max()+.1)),
    
    PCA_2_bound=widgets.FloatRangeSlider( min=pcaData.PCA_1.min()-.1,max=pcaData.PCA_1.max()+.1,
                                   value=(pcaData.PCA_1.min()-.1,pcaData.PCA_1.max()+.1)),
    
    PCA_3_bound=widgets.FloatRangeSlider( min=pcaData.PCA_3.min()-.1,max=pcaData.PCA_3.max()+.1,
                                   value=(pcaData.PCA_3.min()-.1,pcaData.PCA_3.max()+.1)),
    
    PCA_4_bound=widgets.FloatRangeSlider( min=pcaData.PCA_4.min(),max=pcaData.PCA_4.max(),
                                   value=(pcaData.PCA_4.min(),pcaData.PCA_4.max())),
    xCol=widgets.Dropdown(options=pcaData.columns[3:],value='X'),
    yCol=widgets.Dropdown(options=pcaData.columns[3:],value='Y'),
    zCol=widgets.Dropdown(options=pcaData.columns[3:],value='Z'),
    verbose=widgets.ToggleButton(value=False)
    ):
    kwargDict=dict(locals())
    if verbose:
        print(kwargDict.keys(),kwargDict.values())
    kwargDict=dict(locals())
    kwargKeys=np.array(list(kwargDict.keys()))[:13]
    if verbose:
        for kwargKey in kwargKeys:
            print(kwargKey,end="")
            print(len(kwargDict[kwargKey]),kwargDict[kwargKey][0],kwargDict[kwargKey][1])
    queryStr=' and '.join(
        ['({factorKey:s} >= {factorMin:f}) and ({factorKey:s} <= {factorMax:f})'.format(
            factorKey='_'.join(kwargKey.split('_')[:-1]),
            factorMin=kwargDict[kwargKey][0],
            factorMax=kwargDict[kwargKey][1]) \
         for kwargKey in kwargKeys])
    if verbose:
        print("query:",queryStr)
        print("selection shape:",pcaData.query(queryStr).shape)
    go=ply.graph_objs
    fig=go.Figure(
        data=[
                go.Scatter3d(x=pcaData.query('AtomType == "POT"')[xCol],
                             y=pcaData.query('AtomType == "POT"')[yCol],
                             z=pcaData.query('AtomType == "POT"')[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color='grey',
                                         opacity=.0625)),
                go.Scatter3d(x=pcaData.query(queryStr)[xCol],
                             y=pcaData.query(queryStr)[yCol],
                             z=pcaData.query(queryStr)[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color='blue',
                                         opacity=.25)),
                go.Scatter3d(
                    x=pcaData.query(
                            '(AtomType == "CA")'
                        )[xCol],
                    y=pcaData.query(
                            '(AtomType == "CA")'
                        )[yCol],
                    z=pcaData.query(
                            '(AtomType == "CA")'
                        )[zCol],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25))
             ],
        layout=dict(
            width=800,height=600)
    )
    fig.show()

interactive(children=(FloatRangeSlider(value=(-0.0937, 196.57999999999998), description='X_bound', max=196.579…

# Cylindrical Filter

In [309]:
import math
tqdm.tqdm.pandas(tqdm.tqdm_notebook)

In [292]:
diffCoords.query('AtomType=="CA"').head()

,Frame,AtomID,AtomType,X,Y,Z,D_958,D_1178,D_1366,D_2377,D_3796
0,1,958,CA,123.2928,110.9940,58.3107,0.000000,92.177611,41.136629,70.165424,63.568180
1,1,1178,CA,76.6708,159.4728,121.3417,92.177611,0.000000,96.855696,79.014331,72.235503
2,1,1366,CA,95.1964,126.0866,32.3294,41.136629,96.855696,0.000000,51.106721,51.621998
3,1,2377,CA,53.3092,115.8309,59.7553,70.165424,79.014331,51.106721,0.000000,75.214944
4,1,3796,CA,103.7853,171.4280,55.4642,63.568180,72.235503,51.621998,75.214944,0.000000


In [302]:
diffCoords=diffCoords.drop(columns=[colName for colName in diffCoords.columns if 'Center' in colName])
diffCoords=diffCoords.reset_index()
diffCoords.head()

,Frame,AtomID,AtomType,X,Y,Z,D_958,D_1178,D_1366,D_2377,D_3796
0,1,958,CA,123.2928,110.9940,58.3107,0.000000,92.177611,41.136629,70.165424,63.568180
1,1,1178,CA,76.6708,159.4728,121.3417,92.177611,0.000000,96.855696,79.014331,72.235503
2,1,1366,CA,95.1964,126.0866,32.3294,41.136629,96.855696,0.000000,51.106721,51.621998
3,1,2377,CA,53.3092,115.8309,59.7553,70.165424,79.014331,51.106721,0.000000,75.214944
4,1,3796,CA,103.7853,171.4280,55.4642,63.568180,72.235503,51.621998,75.214944,0.000000


In [339]:
cylCenters=diffCoords.query('AtomType=="CA"')[['Frame','X','Y','Z']].groupby(
    ['Frame']).progress_aggregate(np.mean)
diffCyl=diffCoords.set_index(['Frame']).join(
    cylCenters,rsuffix='_Center',how='outer')
diffCyl=diffCyl.reset_index()
diffCyl['R']=np.sqrt(
        (diffCyl['X']-diffCyl['X_Center'])**2 + \
        (diffCyl['Y']-diffCyl['Y_Center'])**2)
diffCyl['Theta']=np.arctan2(diffCyl['Y']-diffCyl['Y_Center'],diffCyl['X']-diffCyl['X_Center'])
diffCyl=diffCyl[np.concatenate([
    diffCyl.columns[:6],
    [colName for colName in diffCyl.columns if '_Center' in colName],
    ['R','Theta'],
    [colName for colName in diffCyl.columns if 'D_' in colName],
])]
print(diffCyl.shape)
diffCyl.head()


100%|██████████| 2081/2081 [00:00<00:00, 20804.76it/s]

(43701, 16)


,Frame,AtomID,AtomType,X,Y,Z,X_Center,Y_Center,Z_Center,R,Theta,D_958,D_1178,D_1366,D_2377,D_3796
0,1,958,CA,123.2928,110.9940,58.3107,90.4509,136.76246,65.44026,41.744508,-0.665293,0.000000,92.177611,41.136629,70.165424,63.568180
1,1,1178,CA,76.6708,159.4728,121.3417,90.4509,136.76246,65.44026,26.564087,2.116184,92.177611,0.000000,96.855696,79.014331,72.235503
2,1,1366,CA,95.1964,126.0866,32.3294,90.4509,136.76246,65.44026,11.683054,-1.152519,41.136629,96.855696,0.000000,51.106721,51.621998
3,1,2377,CA,53.3092,115.8309,59.7553,90.4509,136.76246,65.44026,42.633743,-2.628399,70.165424,79.014331,51.106721,0.000000,75.214944
4,1,3796,CA,103.7853,171.4280,55.4642,90.4509,136.76246,65.44026,37.141700,1.203585,63.568180,72.235503,51.621998,75.214944,0.000000


## Explore radial cutoff

In [340]:
@interact_manual
def explore_milestone_bounds( #'D_958', 'D_1178', 'D_1366', 'D_2377', 'D_3796'
    R_bound=widgets.FloatRangeSlider(     min=0,max=diffCyl.R.max()+.1,
                                   value=(0,diffCyl.R.max()+.1)),
    
    Theta_bound=widgets.FloatRangeSlider(     min=-np.pi,max=np.pi,
                                   value=(-np.pi,np.pi)),
    
    Z_bound=widgets.FloatRangeSlider(     min=diffCyl.Z.min()-.1,max=diffCyl.Z.max()+.1,
                                   value=(diffCyl.Z.min()-.1,diffCyl.Z.max()+.1)),
    xCol=widgets.Dropdown(options=diffCyl.columns[3:],value='X'),
    yCol=widgets.Dropdown(options=diffCyl.columns[3:],value='Y'),
    zCol=widgets.Dropdown(options=diffCyl.columns[3:],value='Z'),
    verbose=widgets.ToggleButton(value=False)
    ):
    kwargDict=dict(locals())
    if verbose:
        print(kwargDict.keys(),kwargDict.values())
    kwargDict=dict(locals())
    kwargKeys=np.array(list(kwargDict.keys()))[:3]
    if verbose:
        for kwargKey in kwargKeys:
            print(kwargKey,end="")
            print(len(kwargDict[kwargKey]),kwargDict[kwargKey][0],kwargDict[kwargKey][1])
    queryStr=' and '.join(
        ['({factorKey:s} >= {factorMin:f}) and ({factorKey:s} <= {factorMax:f})'.format(
            factorKey='_'.join(kwargKey.split('_')[:-1]),
            factorMin=kwargDict[kwargKey][0],
            factorMax=kwargDict[kwargKey][1]) \
         for kwargKey in kwargKeys])
    if verbose:
        print("query:",queryStr)
        print("selection shape:",diffCyl.query(queryStr).shape)
        print(diffCyl.query(queryStr).head())
    go=ply.graph_objs
    fig=go.Figure(
        data=[
                go.Scatter3d(x=diffCyl.query('AtomType == "CA"')['X_Center'],
                             y=diffCyl.query('AtomType == "CA"')['Y_Center'],
                             z=diffCyl.query('AtomType == "CA"')['Z_Center'],
                             mode='markers',
                             marker=dict(size=12,
                                         color='red',
                                         opacity=.5)),
                go.Scatter3d(x=diffCyl.query('AtomType == "POT"')[xCol],
                             y=diffCyl.query('AtomType == "POT"')[yCol],
                             z=diffCyl.query('AtomType == "POT"')[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color='grey',
                                         opacity=.0625)),
                go.Scatter3d(x=diffCyl.query(queryStr)[xCol],
                             y=diffCyl.query(queryStr)[yCol],
                             z=diffCyl.query(queryStr)[zCol],
                             mode='markers',
                             marker=dict(size=2,
                                         color='blue',
                                         opacity=.25)),
                go.Scatter3d(
                    x=diffCyl.query(
                            '(AtomType == "CA")'
                        )[xCol],
                    y=diffCyl.query(
                            '(AtomType == "CA")'
                        )[yCol],
                    z=diffCyl.query(
                            '(AtomType == "CA")'
                        )[zCol],
                    mode='markers',
                    marker=dict(size=12,
                                color='black',
                                opacity=.25))
             ],
        layout=dict(
            width=800,height=600)
    )
    fig.show()

interactive(children=(FloatRangeSlider(value=(0.0, 171.15737440875327), description='R_bound', max=171.1573744…